In [44]:
import tensorflow as tf
import numpy as np
import skimage.io as image
from skimage.transform import resize
print (tf.__version__)

2.1.0-dev20191121


In [97]:
style_image = image.imread('./images/monet.jpg')
content_image = image.imread('./images/louvre.jpg')

content_image.shape

(600, 800, 3)

In [98]:
def reshape_and_normalize_image(image):
    """
    Reshape and normalize the input image (content or style)
    """
    
    # Reshape image to mach expected input of VGG16
    image = np.reshape(image, ((1,) + image.shape))
    MEANS = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3)) 
    
    # Substract the mean to match the expected input of VGG16
    image = image - MEANS
    
    return image

In [99]:
content_image = resize(content_image, (244, 244))
content_image = reshape_and_normalize_image(content_image)
# content_image.shape

style_image = resize(style_image, (244, 244))
style_image = reshape_and_normalize_image(style_image)


m, n_h, n_W, n_C = content_image.shape
# content_image = tf.reshape(content_image, shape=[m, n_h, n_W, n_C])
content_image

array([[[[-123.58196078, -116.54370588, -103.48409804],
         [-123.58196078, -116.53978431, -103.47853713],
         [-123.58109316, -116.53226288, -103.46641857],
         ...,
         [-123.59086467, -116.59574703, -103.57853739],
         [-123.58980392, -116.59574703, -103.58213725],
         [-123.58980392, -116.59860784, -103.58213725]],

        [[-123.58196078, -116.54360286, -103.48399502],
         [-123.57989118, -116.53947526, -103.47461556],
         [-123.57803922, -116.52920894, -103.45902572],
         ...,
         [-123.58758678, -116.59076471, -103.57133766],
         [-123.58980392, -116.59437722, -103.57398506],
         [-123.58632578, -116.5980614 , -103.57429412]],

        [[-123.58196078, -116.53978431, -103.48101221],
         [-123.57803922, -116.53586275, -103.47069399],
         [-123.57803922, -116.52528737, -103.45510415],
         ...,
         [-123.58980392, -116.58684314, -103.57037255],
         [-123.5869439 , -116.59076471, -103.57037255],
  

In [117]:
def generate_noise_image(content_image, noise_ratio = 0.6):
    """
    Generates a noisy image by adding random noise to the content_image
    """
    
    IMAGE_WIDTH = 244
    IMAGE_HEIGHT = 244
    COLOR_CHANNELS = 3
    
    # Generate a random noise_image
    noise_image = np.random.uniform(-20, 20, (1, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS)).astype('float32')
    
    # Set the input_image to be a weighted average of the content_image and a noise_image
    input_image = noise_image * noise_ratio + content_image * (1 - noise_ratio)
    
    return input_image

In [ ]:
generated_image = generate_noise_image(content_image)

In [100]:
model = tf.keras.applications.vgg19.VGG19(weights='imagenet', include_top=False, input_tensor=tf.keras.Input(shape=(224, 224, 3)))
model.trainable = False
for layer in model.layers:
    print(layer.name)

input_2
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool


In [119]:
content_layers = ['block4_conv2']

style_layers = [('block1_conv1', 0.2), 
                ('block2_conv1', 0.2), 
                ('block3_conv1', 0.2), 
                ('block4_conv1', 0.2), 
                ('block5_conv1', 0.2)]
style_layers = ['block1_conv1', 
                'block2_conv1', 
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']


content_outputs = [model.get_layer(name).output for name in content_layers]
style_outputs = [model.get_layer(name).output for name in style_layers]

In [122]:
content_input = tf.keras.applications.vgg19.preprocess_input(content_image)
style_input = tf.keras.applications.vgg19.preprocess_input(style_image)
generated_input = tf.keras.applications.vgg19.preprocess_input(generated_image)

a_content = tf.keras.Model([model.input], outputs = content_outputs, name='VGG19')
a_style = tf.keras.Model(inputs=[model.input], outputs = style_outputs, name='VGG19')

# Defining Support Functions

## Compute Cost

In [104]:
def compute_content_cost(a_C, a_G):
    
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    
    a_C_unrolled = tf.reshape(a_C, shape=[m, n_H*n_W, n_C])
    a_G_unrolled = tf.reshape(a_G, shape=[m, n_H*n_W, n_C])
    
    content_cost = 1/(4*n_H*n_W*n_C) * tf.reduce_sum(tf.square(a_C_unrolled - a_G_unrolled))
    
    return content_cost

In [141]:
# TEST

tf.random.set_seed(1)
a_C = tf.random.normal([1, 4, 4, 3], mean=1, stddev=4)
a_G = tf.random.normal([1, 4, 4, 3], mean=1, stddev=4)
J_content = compute_content_cost(a_C, a_G)
print("J_content = \n" + str(J_content.numpy()))

J_content = 
7.056877


## Style Cost

In [23]:
def gram_matrix(A):
    return tf.matmul(A, tf.transpose(A))

In [140]:
# TEST

tf.random.set_seed(1)
A = tf.random.normal([3, 2*1], mean=1, stddev=4)
GA = gram_matrix(A)
print("GA = \n" + str(GA.numpy()))

GA = 
[[ 63.1888    -26.721275   -7.7320204]
 [-26.721275   12.76758    -2.5158243]
 [ -7.7320204  -2.5158243  23.752384 ]]


In [24]:
def compute_style_cost_layer(a_S, a_G):
    
    m, n_H, n_W, n_C = a_G.get_shape().as_list()
    
    a_S_unrolled = tf.reshape(tf.transpose(a_S, perm=[0, 3, 1, 2]), shape=[n_C, n_H*n_W])
    a_G_unrolled = tf.reshape(tf.transpose(a_G, perm=[0, 3, 1, 2]), shape=[n_C, n_H*n_W])
    
    GS = gram_matrix(a_S_unrolled)
    GG = gram_matrix(a_G_unrolled)
    
    layer_style_cost = 1/(4*(n_C**2)*(n_H*n_W)**2) * tf.reduce_sum(tf.square(GS - GG))
    return layer_style_cost

In [144]:
# TEST

tf.random.set_seed(1)
a_S = tf.random.normal([1, 4, 4, 3], mean=1, stddev=4)
a_G = tf.random.normal([1, 4, 4, 3], mean=1, stddev=4)
J_style_layer = compute_style_cost_layer(a_S, a_G)

print("J_style_layer = " + str(J_style_layer.numpy()))

J_style_layer = 14.017808


In [127]:
def compute_style_cost(style_input, generated_input, style_layers, coefs):
    
    style_cost = 0
    style_outputs = a_style(style_input)
    generated_outputs = a_style(generated_input)
    
    for i in range(0, len(style_outputs)):
        
        a_S = style_outputs[i]
        a_G = generated_outputs[i]
        
        layer_cost = compute_style_cost_layer(a_S, a_G)
        style_cost += (coefs[i] * layer_cost)
    
    return style_cost

## Total Cost

In [128]:
def total_cost(content_cost, style_cost, alpha = 10, beta = 10):
    total_cost = (alpha * content_cost) + (beta * style_cost)
    return total_cost

In [146]:
# TEST

np.random.seed(3)
J_content = np.random.randn()    
J_style = np.random.randn()
J = total_cost(J_content, J_style)
print("J = " + str(J))

J = 22.251383239423077


# Building Model

In [151]:
content_output = a_content(content_input)
generated_output = a_content(generated_input)

J_content = compute_content_cost(content_output, generated_output)
print("\nJ_content = " + str(J_cost.numpy()))
# style_output = a_style(style_input)


J_content = 371415.31348055525


In [153]:
coefs = [0.2, 0.2, 0.2, 0.2, 0.2]
J_style = compute_style_cost(style_input, generated_input, style_layers, coefs)
print("\nJ_style = " + str(J_style.numpy()))


J_style = 1258364511512.917


In [155]:
J_total = total_cost(J_content, J_style)
print("J_total = " + str(J_total.numpy()))

J_total = 12583648829282.305
